In [2]:
import numpy as np
import pandas as pd
import random
import math
import matplotlib as plt
import itertools

In [35]:
def mmn_queueing_jsq(n,arrival_rate,service_rate,simulation_time=60,simuseed=math.nan):
    '''
    Simulating the mmn queueing system using the method of JSQ 
    (namely join the shortest queue method), the user needs to provide
    the number of servers (n),arrival rate (lambda) and service rate (mu),
    in case of n >= 2, if mu is provided as a scaler, then all servers
    share the same mu, if mu is provided as a vector, then all servers may have different mu,
    the simulation time is by default 60 units, and for reproducibility purposes, the user
    can define simulation seed, by default the seed value is not set.
    
    n:               the number of servers, a positive finite integer
    arrival_rate:    exponential rate of arrival
    service_rate:    exponential rate of service time
    simulation_time: the simulation time, a positive finite number
    simuseed:        the simulation seed number
    
    The output will xxx
    '''
    
    if math.isnan(simuseed) == True:
        pass
    else:
        random.seed(simuseed)

    if not type(n) is int:
        raise TypeError("n needs to be integer")
    if n < 0:
        raise Exception("n needs to be positive")
    
    lamb =  arrival_rate
    
    if isinstance(service_rate, list):
        if len(service_rate) == n:
            mu = service_rate
        elif (n > 1 and len(service_rate) == 1):
            mu = n*service_rate
        else:
            raise Exception("Unmatched cardinality for service rate and n")
    elif isinstance(service_rate, int):
        if n == 1:
            mu = [service_rate]
        elif n > 1:
            mu = n*[service_rate]
    else:
        raise Exception("Please use int or list type for service rate")
    
    
    # Simulating the event calendar
    table_col = np.arange(n)*4+3
    avail_col = np.arange(n)*4+4
    queue_col = np.arange(n)*4+5
    total_col = np.arange(n)*4+6
    compare_col = np.insert(avail_col,0,[1,2])
    
    event_calendar = np.zeros([2,3+4*n])
    event_calendar[-1,1] = random.expovariate(lamb)
    event_calendar[-1,2] = simulation_time
    event_calendar[-1,avail_col] = math.nan
    
    while event_calendar[-1,0] < simulation_time:
        event_calendar = np.vstack([event_calendar,event_calendar[-1,:]])
        event_calendar[-1,0] = min(event_calendar[-2,compare_col])
        
        # If the next event is an arrival
        if event_calendar[-1,0] == event_calendar[-2,1]:
            event_calendar[-1,1] = event_calendar[-1,0]+random.expovariate(lamb)
            server_assignment = random.choice(np.where(event_calendar[-1,total_col] == \
                                   min(event_calendar[-1,total_col]))[0])
            
            # If the server is not occupied
            if event_calendar[-1,table_col[server_assignment]] == 0:
                event_calendar[-1,table_col[server_assignment]] += 1
                event_calendar[-1,avail_col[server_assignment]] = event_calendar[-1,0] + \
                                                        random.expovariate(mu[server_assignment])
                event_calendar[-1,total_col[server_assignment]] += 1
            
            # Else if the server is occupied
            elif event_calendar[-1,table_col[server_assignment]] == 1:
                event_calendar[-1,queue_col[server_assignment]] += 1
                event_calendar[-1,total_col[server_assignment]] += 1
            
        # Else if the next event is the simulation termination
        elif event_calendar[-1,0] == simulation_time:
            pass
        
        # Else if the next event is a service completion
        else:
            server_completion = np.where(event_calendar[-2,avail_col] == event_calendar[-1,0])[0][0]
            
            # If there is no queue behind
            if event_calendar[-1,queue_col[server_completion]] == 0:
                event_calendar[-1,table_col[server_completion]] -= 1
                event_calendar[-1,avail_col[server_completion]] = math.nan
                event_calendar[-1,total_col[server_completion]] -= 1
            
            # Else if there is a queue behind
            elif event_calendar[-1,queue_col[server_completion]] > 0:
                event_calendar[-1,avail_col[server_completion]] = event_calendar[-1,0] + \
                                                        random.expovariate(mu[server_completion])
                event_calendar[-1,queue_col[server_completion]] -= 1
                event_calendar[-1,total_col[server_completion]] -= 1
    
    event_calendar = pd.DataFrame(event_calendar).tail(-1)
    event_calendar.columns = ['Time','Next Customer','Finish Time'] + \
                    list(itertools.chain.from_iterable([[f"Server{i}", \
                    f"Server{i} Available Time",f"Server{i} Queue",f"Server{i} Total"] for i in range(1,n+1)]))
    
    # Concluding performance measures
    pm_drop_cols = np.concatenate([np.array([1,2]), avail_col], axis=0)
    pm = event_calendar.drop(event_calendar.iloc[:,pm_drop_cols], axis=1)
    
    return event_calendar


In [190]:
mmn_queueing_jsq(2,10,[19,26],simulation_time=5,simuseed=8).head(20)

,Time,Next Customer,Finish Time,Server1,Server1 Available Time,Server1 Queue,Server2,Server2 Available Time,Server2 Queue
1,0.000000,0.025710,5.0,0.0,NaN,0.0,0.0,NaN,0.0
2,0.025710,0.353506,5.0,1.0,0.037003,0.0,0.0,NaN,0.0
3,0.037003,0.353506,5.0,0.0,NaN,0.0,0.0,NaN,0.0
4,0.353506,0.357982,5.0,1.0,0.368468,0.0,0.0,NaN,0.0
5,0.357982,1.062516,5.0,1.0,0.368468,0.0,1.0,0.377675,0.0
6,0.368468,1.062516,5.0,0.0,NaN,0.0,1.0,0.377675,0.0
7,0.377675,1.062516,5.0,0.0,NaN,0.0,0.0,NaN,0.0
8,1.062516,1.065593,5.0,0.0,NaN,0.0,1.0,1.085730,0.0
9,1.065593,1.133909,5.0,1.0,1.185399,0.0,1.0,1.085730,0.0
10,1.085730,1.133909,5.0,1.0,1.185399,0.0,0.0,NaN,0.0


In [37]:
mmn_queueing_jsq(2,10,[19,26],simulation_time=5,simuseed=8).tail(20)

,Time,Next Customer,Finish Time,Server1,Server1 Available Time,Server1 Queue,Server1 Total,Server2,Server2 Available Time,Server2 Queue,Server2 Total
85,4.070835,4.416096,5.0,0.0,NaN,0.0,0.0,0.0,NaN,0.0,0.0
86,4.416096,4.438468,5.0,1.0,4.452284,0.0,1.0,0.0,NaN,0.0,0.0
87,4.438468,4.502559,5.0,1.0,4.452284,0.0,1.0,1.0,4.456921,0.0,1.0
88,4.452284,4.502559,5.0,0.0,NaN,0.0,0.0,1.0,4.456921,0.0,1.0
89,4.456921,4.502559,5.0,0.0,NaN,0.0,0.0,0.0,NaN,0.0,0.0
90,4.502559,4.514039,5.0,0.0,NaN,0.0,0.0,1.0,4.542708,0.0,1.0
91,4.514039,4.641103,5.0,1.0,4.580196,0.0,1.0,1.0,4.542708,0.0,1.0
92,4.542708,4.641103,5.0,1.0,4.580196,0.0,1.0,0.0,NaN,0.0,0.0
93,4.580196,4.641103,5.0,0.0,NaN,0.0,0.0,0.0,NaN,0.0,0.0
94,4.641103,4.704836,5.0,0.0,NaN,0.0,0.0,1.0,4.706308,0.0,1.0


In [27]:
ahgb = mmn_queueing_jsq(2,10,[19,26],simulation_time=5,simuseed=8)
ahgb

,Time,Server1,Server1 Queue,Server2,Server2 Queue
1,0.000000,0.0,0.0,0.0,0.0
2,0.025710,1.0,0.0,0.0,0.0
3,0.037003,0.0,0.0,0.0,0.0
4,0.353506,1.0,0.0,0.0,0.0
5,0.357982,1.0,0.0,1.0,0.0
...,...,...,...,...,...
100,4.824693,1.0,0.0,0.0,0.0
101,4.825154,1.0,0.0,1.0,0.0
102,4.880737,0.0,0.0,1.0,0.0
103,4.973112,0.0,0.0,0.0,0.0


In [24]:
ahgb["sbd"] = 1

In [172]:
np.array([3,2]) + np.array([1,10])

array([ 4, 12])

In [34]:
np.insert(np.arange(3)*4+4,0,[1,2])

array([ 1,  2,  4,  8, 12])